In [24]:
from utils.processor import *
from utils.solver import *
from config import *
from utils.tom_api import *

In [25]:
data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')
origin = (HOME['lat'], HOME['lon'])
# dest = (PACIFIC_CENTER['lat'], PACIFIC_CENTER['lon'])
ttapi = TomSearchAPI(API_KEY)
dest = ttapi.find_location('Robson Square')
print(dest)
dest = dest['location']
dest = (dest['lat'], dest['lon'])
dest

{'poi_name': 'UBC Robson Square', 'address': '800 Robson Street, Vancouver BC V6Z', 'location': {'lat': 49.2825, 'lon': -123.1207}}


(49.2825, -123.1207)

In [26]:
topK = 10
stay_duration = 1.5 * 60 
parking_budget = 15.00

processor = Processor(datapath, origin, dest, topK, stay_duration, parking_budget)
variables = processor.form_variables_dict()
variables

(7954, 21)
Original #of separate twins: 6086
% retained after merging: 49.90141307919816


/Users/ahnafayub/Desktop/Programming/MATH-340/utils/processor.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_meter['METERID'] = (twins.loc[first]['METERID'], twins.loc[second]['METERID'])


*** Data shape after merge: (3655, 24) ***
(3655, 25)
IDX - 0
Time limit: 2 	 Rate: 5.0
*** Travel time: 23.083333333333332 ***
out: 2 - in: 0
~~~ Parking Status: 3 ~~~
{'b_1': 1.0, 'constraint_coeffs': {'parking_fee': 7.5, 'savings': 7.5, 'expected_avail': 3, 'et_next_out': 11.0, 'et_next_in': 24.0, 'walk_time': 2.733, 'travel_et': 23.083333333333332}, 'meterid': '630833', 'latln': (49.28193539343345, -123.12298498208641)}
IDX - 1
Time limit: 2 	 Rate: 3.0
******************** Twin **********************
*** Travel time: 22.866666666666667 ***
out: 1 - in: 1
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 2
Time limit: 2 	 Rate: 4.0
******************** Twin **********************
*** Travel time: 23.333333333333332 ***
out: 1 - in: 0
~~~ Parking Status: 3 ~~~
{'b_2': 1.0, 'constraint_coeffs': {'parking_fee': 6.0, 'savings': 9.0, 'expected_avail': 3, 'et_next_out': 12.0, 'et_next_in': 24.0, 'walk_time': 3.033, 'travel_et': 23.333333333333332}, 'meterid': ('640721',

{'y_1': {'b_1': 1.0,
  'constraint_coeffs': {'parking_fee': 7.5,
   'savings': 7.5,
   'expected_avail': 3,
   'et_next_out': 11.0,
   'et_next_in': 24.0,
   'walk_time': 2.733,
   'travel_et': 23.083333333333332},
  'meterid': '630833',
  'latln': (49.28193539343345, -123.12298498208641)},
 'y_2': {'b_2': 1.0,
  'constraint_coeffs': {'parking_fee': 6.0,
   'savings': 9.0,
   'expected_avail': 3,
   'et_next_out': 12.0,
   'et_next_in': 24.0,
   'walk_time': 3.033,
   'travel_et': 23.333333333333332},
  'meterid': ('640721', '640719'),
  'latln': (49.28361049869213, -123.12253957200025)},
 'y_3': {'b_3': 1.0,
  'constraint_coeffs': {'parking_fee': 6.0,
   'savings': 9.0,
   'expected_avail': 2,
   'et_next_out': 11.0,
   'et_next_in': 22.0,
   'walk_time': 2.883,
   'travel_et': 23.233333333333334},
  'meterid': '640723',
  'latln': (49.28352530367541, -123.1226683511258)},
 'y_4': {'b_4': 1.0,
  'constraint_coeffs': {'parking_fee': 4.5,
   'savings': 10.5,
   'expected_avail': 3,
   '

In [27]:
solver = Solver(variables)
solver.solve_LP()
soln = solver.get_soln_summary()
soln

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/5c5fad5f9c1c4e1ab33d561c144493af-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/5c5fad5f9c1c4e1ab33d561c144493af-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 215 RHS
At line 221 BOUNDS
At line 256 ENDATA
Problem MODEL has 5 rows, 34 columns and 170 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 34 (0) columns and 170 (0) elements
Perturbing problem by 0.001% of 1.1666667 - largest nonzero change 2.1227554e-05 ( 0.0019290484%) - largest zero change 0
0  Obj 0 Primal inf 1.4093135 (2)
1  Obj 0.71324344
Optimal - objective value 0.71323529
Optimal objective 0.7

,var,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
29,y_30,0.713235,1.5,4,22.266667,5.033,11.0,25.0
31,y_32,0.000000,4.5,3,21.366667,5.633,15.0,24.0
26,y_27,0.000000,4.5,3,21.450000,5.317,14.0,23.0
27,y_28,0.000000,4.5,3,21.483333,5.117,13.0,22.0
22,y_23,0.000000,4.5,2,21.483333,5.167,13.0,22.0
19,y_20,0.000000,4.5,3,21.533333,4.950,13.0,23.0
17,y_18,0.000000,4.5,3,21.600000,4.750,12.0,24.0
21,y_22,0.000000,4.5,3,21.616667,4.733,11.0,24.0
15,y_16,0.000000,4.5,2,21.666667,4.600,20.0,25.0
8,y_9,0.000000,10.5,3,21.783333,4.233,14.0,25.0


# Visualization

In [28]:
import folium
fmap = folium.Map(location=dest, zoom_start=16, width="100%", height="50%")

# folium.Marker(
#     location=origin, icon=folium.Icon(color="red",icon="home", prefix='fa')
# ).add_to(fmap)

folium.Marker(
    location=dest, icon=folium.Icon(color="red",icon="location-dot", prefix='fa')
).add_to(fmap)


for i, spot in enumerate(soln.loc[soln.optimal_weight > 0]['var']):
    print(spot)
    rank = i+1
    spot_loc = variables[spot]['latln']
    folium.Marker(
        location = spot_loc,
        popup = f'Rank-{rank}',
        icon = folium.Icon(color="blue", icon="circle-parking", prefix='fa')
    ).add_to(fmap)
    
# for r_id, result in enumerate(res['results']):
#     folium.Marker(
#         location = [result['position']['lat'], result['position']['lon']],
# #         popup = result['poi']['name'],
#         popup = generate_popup(r_id, result),
#         icon = get_icon(poi_category)
#     ).add_to(fmap)

fmap

y_30
